# Process Heterostructure Systems
____

# Import Modules

In [1]:
import sys
import os

import pickle

import numpy as np

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

from pymatgen.io.ase import AseAtomsAdaptor

from ase.visualize import view
from ase import io

# #############################################################################
# #############################################################################

from methods import process_atom, get_unique_str_repr

# Script Inputs

In [2]:
max_thickness = 4
z_vacuum = 13
frac_constr_supp = 0.6

# Load Data

In [3]:
df = pickle.load(
    open(
        os.path.join(
#             "dataframe_short.pickle",

            os.environ["PROJ_DATA"],
            "01_fe_graph_proj/04_lattice_matching/181218_data",
            "job_dataframe.pickle",
            ),
        "rb"
        )
    )

# Replace NaN with '[]' (empty lists)
for row in df.loc[df.sys_data_i.isnull(), 'sys_data_i'].index:
    df.at[row, 'sys_data_i'] = []

# Unpack heterointerface column into longer dataframe

In [4]:
master_data_list = []
for i_cnt, row_i in df.iterrows():
    dict_i = row_i.to_dict()
    dict_i.pop("sys_data_i")

    for sys_j in row_i["sys_data_i"]:
        dict_j = {**dict_i, **sys_j}
        atoms_i = AseAtomsAdaptor.get_atoms(sys_j["heterointerface"])

        atoms_processed_i = process_atom(
            atoms_i,
            max_thickness=max_thickness,
            z_vacuum=z_vacuum,
            frac_constr_supp=frac_constr_supp,
            )

        dict_j["atoms"] = atoms_processed_i

        master_data_list.append(dict_j)

# Instantiate the dataframe
df_2 = pd.DataFrame(master_data_list)

Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is 

# Remove duplicate structures

In [5]:
def get_area(row_i):
    area = np.linalg.norm(np.cross(
        row_i["atoms"].cell[0],
        row_i["atoms"].cell[1],
        ))

    return(area)

df_2["area_2"] = df_2.apply(
    get_area,
    axis=1,
    )


df_2["uniq_str_repr"] = df_2.apply(
    get_unique_str_repr,
    axis=1,
    )

df_3 = df_2.drop_duplicates(subset="uniq_str_repr")

# Removing "long" heterostructures

In [6]:
def structure_square_enough(row_i):
    """
    """
    min_ind = np.argmin([np.linalg.norm(i) for i in row_i["uv1"]])

    ind_list = [0, 1]
    ind_list.remove(min_ind)

    uv_small = row_i["uv1"][min_ind]
    uv_large = row_i["uv1"][ind_list[0]]

    len_u_large = np.linalg.norm(uv_large)
    len_u_small = np.linalg.norm(uv_small)
    large_small_ratio = len_u_large / len_u_small

    if large_small_ratio < 3.:
        return(True)
    else:
        return(False)

In [7]:
df_3["structure_square_enough"] = df_3.apply(
    structure_square_enough,
    axis=1,
    )

df_3 = df_3[df_3["structure_square_enough"]]

/home/raul_desktop/anaconda2/envs/MPInterface/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Getting rid of structures with too many metal atoms

In [9]:
def num_m_atoms(row_i):
    """
    """
    atoms_i = row_i["atoms"]

    num_m_atoms = len([i for i in atoms_i.get_atomic_numbers() if i != 6])

    return(num_m_atoms)

In [10]:
df_3["num_m_atoms"] = df_3.apply(
    num_m_atoms,
    axis=1,
    )

df_3 = df_3[df_3["num_m_atoms"] <= 60]

# Save processed data frame

In [39]:
pickle.dump(
    df_3,
    open(
        "processed_dataframe.pickle",
        "wb"
        )
    )